LSTM Model
1. Make sure all packages are installed (WINDOWS)

*    pip install numpy
*   pip install pandas
*   pip install os-sys
*    pip install matplotlib
*   pip install seaborn
*   pip install scikit-learn
*   pip install keras
*   pip install tensorflow 

1. Make sure all packages are installed (MACS AND/OR ANACONDA ENVIRONMENT)
*   conda  install numpy
*   conda  install pandas
*   conda  install os-sys
*   conda  install matplotlib
*   conda  install seaborn
*   conda  install scikit-learn
*   conda  install keras
*   conda  install tensorflow 

Links:

Kaggle Data: https://www.kaggle.com/harnoor343/fall-detection-accelerometer-data 

Potential Issues:
 * can I reshape before putting in LSTM layer :(

Todo:
* test script and sensordata

In [ ]:
#DO NOT RUN IF YOU ARE NOT USING WINDOWS AND/OR NOT USING ANACONDA
%pip install numpy
%pip install pandas
%pip install os-sys
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install keras
%pip install tensorflow

In [36]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import sklearn.model_selection

#Yeah... that's all
run()


DeviceOrientation
AccelerationX
AccelerationY
AccelerationZ
Label
AccelerationX_fft
AccelerationY_fft
AccelerationZ_fft
[0 1 3 2 4 5]
(141819, 7) (60780, 7, 1) (141819, 1) (60780, 1)


C:\Users\siyuz\AppData\Local\Temp\ipykernel_5892\2931986576.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x = np.array(df.drop(['Label'], 1)) #drops row 'Label'


Epoch 1/10
4432/4432 [==============================] - 28s 6ms/step - loss: 1.1252 - accuracy: 0.5552 - val_loss: 0.8920 - val_accuracy: 0.6594
Epoch 2/10
4432/4432 [==============================] - 27s 6ms/step - loss: 0.8413 - accuracy: 0.6752 - val_loss: 0.7880 - val_accuracy: 0.6932
Epoch 3/10
4432/4432 [==============================] - 27s 6ms/step - loss: 0.7838 - accuracy: 0.6924 - val_loss: 0.7589 - val_accuracy: 0.7018
Epoch 4/10
4432/4432 [==============================] - 27s 6ms/step - loss: 0.7576 - accuracy: 0.7004 - val_loss: 0.7406 - val_accuracy: 0.7065
Epoch 5/10
4432/4432 [==============================] - 27s 6ms/step - loss: 0.7427 - accuracy: 0.7047 - val_loss: 0.7271 - val_accuracy: 0.7090
Epoch 6/10
4432/4432 [==============================] - 27s 6ms/step - loss: 0.7299 - accuracy: 0.7108 - val_loss: 0.7171 - val_accuracy: 0.7157
Epoch 7/10
4432/4432 [==============================] - 27s 6ms/step - loss: 0.7218 - accuracy: 0.7128 - val_loss: 0.7155 - val_ac

INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [37]:


def data_collection():
    # create a new directory called 'kaggle_normalized', ignore it if it already exists
    os.makedirs('./normalized', exist_ok=True)

    labels = ['runfall', 'downSit', 'freeFall', 'runSit', 'walkFall', 'walkSit']
    for label in labels:
        #read all csv files in the directory "archive' + label, delimit by ';'
        files = glob.glob('./archive/' + label + '/*.csv')
        df = pd.concat([pd.read_csv(f, sep=';') for f in files], ignore_index=True)
        # assign label to df column 'Label'
        df['Label'] = label
        # save columns 'DeviceOrientation', 'AccelerationX', 'AccelerationY', 'AccelerationZ', 'Label' to a new csv file
        df[['DeviceOrientation', 'AccelerationX', 'AccelerationY', 'AccelerationZ', 'Label']].to_csv('./normalized/' + label + '.csv', index=False)
        
        
    df = pd.concat([pd.read_csv(f) for f in glob.glob('./normalized/*.csv')], ignore_index = True)
    return df

In [3]:
def data_normalize(df):
    # for each column in 'Accerlation', 'AccelerationY', 'AccelerationZ',
    # add a new column that is the fft of the original column
    for c in ['AccelerationX', 'AccelerationY', 'AccelerationZ']:
        df[c + '_fft'] = np.fft.fft(df[c])

    # debug purposes
    for c in df.columns:
        print(c)
        
    return df
    

In [4]:
def data_plot(df):
    #debugging function. does not need to be called, or return anything

    # plot the first 100 column 'AccelerationX' and 'AccelerationX_fft'
    df[['AccelerationX', 'AccelerationY', 'AccelerationZ']].iloc[:100].plot()

    # Versus Head (the same)
    #df[['AccelerationX_fft', 'AccelerationY_fft', 'AccelerationZ_fft']].head(100).plot()
    df[['AccelerationX', 'AccelerationY', 'AccelerationZ']].head(100).plot()

In [5]:
def data_categorical(df):
    # convert categorical data to numerical data
    df['Label'] = df['Label'].astype('category')
    df['Label'] = df['Label'].cat.codes
    print(df['Label'].unique())

    # convert column `DeviceOrientation` to numerical data
    df['DeviceOrientation'] = df['DeviceOrientation'].astype('category')
    df['DeviceOrientation'] = df['DeviceOrientation'].cat.codes
    return df

In [25]:
def data_split(df):
    x = np.array(df.drop(['Label'], 1)) #drops row 'Label'
    y = np.array(df["Label"])

    #Spliting Data
    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x,y,test_size = 0.3)

    # reshape training and testing data
    y_train = np.array(y_train).reshape(-1,1) # (-1,1) because our data has a single feature, and a 'n' amount of rows
    y_test = np.array(y_test).reshape(-1,1) # (-1,1) bec  ause our data has a single feature, and a 'n' amount of rows
    x_test = np.array(x_test).reshape(x_test.shape[0], x_test.shape[1], 1)

    return x_train, x_test, y_train, y_test

In [26]:
def model_create(x_train):
  model = Sequential()
  model.add(LSTM(128, input_shape=(x_train.shape[1], 1)))
  model.add(Dropout(0.2))
  model.add(Dense(6, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [27]:
def train_and_accurary_model(model, x_train, x_test, y_train, y_test):
  #reshape the features for the LSTM layer (I REALLY WANT TO FIX UGHHHG)
  x_train = np.array(x_train).reshape(x_train.shape[0], x_train.shape[1], 1)

  # train the model
  model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

  # evaluate the model
  test_loss, test_Acc = model.evaluate(x_test, y_test)
  print('Test accuracy:', test_Acc)
  model.summary()
  return model

In [35]:
def model_save(model):
  # save the model
  model.save('saved_model/my_model')
  return model

In [32]:
def run():
    df = data_collection()
    df = data_normalize(df)
    df = data_categorical(df)
    x_train, x_test, y_train, y_test = data_split(df)
    model = model_create(x_train)
    model = train_and_accurary_model(model, x_train, x_test, y_train, y_test)
    model_save(model)

    #debug purposes
    print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)